<b>N_gram</b> are combinations of words in order.

In [5]:
from nltk import ngrams
from nltk.tokenize import word_tokenize

N = 3
sentence = "hi, how are you ?"
tokenized_sentence = word_tokenize(sentence)
n_grams = list(ngrams(tokenized_sentence,N))
for i in n_grams:
    print(i)

('hi', ',', 'how')
(',', 'how', 'are')
('how', 'are', 'you')
('are', 'you', '?')


<i>CountVectorizer</i> and <i>TfidfVectorizer</i> implementations of scikit-learn offers ngrams by ngram_range parameter, which has a minimum and maximum limit.
<p></p>
Now we will implement <b>Logistic Regression</b> with <i>TdidfVectoirize</i> along with <i>N-grams</i> on the IMDB dataset.

In [7]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

if __name__ == "__main__":
    df = pd.read_csv("input/imdb.csv")
    df.sentiment = df.sentiment.apply(lambda x: 1 if x == "positive" else 0)
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.sentiment.values
    kf = model_selection.StratifiedKFold(n_splits=5)
    
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f
        
    for fold_ in range(2):
        # temporary dataframes for train and test
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)
        # initialize TfidfVectorizer with NLTK's word_tokenize
        # function as tokenizer
        tfidf_vec = TfidfVectorizer(
        tokenizer=word_tokenize,
        token_pattern=None,
        ngram_range=(1,3)
        )

        tfidf_vec.fit(train_df.review)

        xtrain = tfidf_vec.transform(train_df.review)
        xtest = tfidf_vec.transform(test_df.review)
  
        model = linear_model.LogisticRegression()
    
        model.fit(xtrain, train_df.sentiment)
        preds = model.predict(xtest)
    
        accuracy = metrics.accuracy_score(test_df.sentiment, preds)
        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")

Fold: 0
Accuracy = 0.8936

Fold: 1
Accuracy = 0.8894

